In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-science-lab-2022-spring-kaggle/train_final.csv
/kaggle/input/data-science-lab-2022-spring-kaggle/test_final.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.metrics import roc_auc_score
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import tree, ensemble
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [4]:
df_train=pd.read_csv('../input/data-science-lab-2022-spring-kaggle/train_final.csv')
df_test=pd.read_csv('../input/data-science-lab-2022-spring-kaggle/test_final.csv')

In [5]:
X=np.array(df_train.drop(['Y','Id'],axis=1))
y=np.array(df_train['Y'])

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
random_state=check_random_state(0)
permutation=random_state.permutation(X.shape[0])
X = X[permutation]
y=y[permutation]
X = X.reshape((X.shape[0], -1))
X=pd.DataFrame(X)
y=pd.DataFrame(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y, random_state=42)

In [7]:
y_train=np.ravel(y_train)
y_test=np.ravel(y_test)

In [8]:
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

13    0.068488
14    0.038228
15    0.018512
7     0.017058
0     0.015676
3     0.011485
12    0.011158
18    0.008649
16    0.007334
6     0.005202
19    0.004999
10    0.004114
20    0.003293
17    0.002473
4     0.002256
21    0.002127
8     0.001813
9     0.001527
23    0.000999
22    0.000412
5     0.000410
1     0.000000
11    0.000000
2     0.000000
dtype: float64

In [9]:
from sklearn.feature_selection import SelectKBest
sel_five_cols = SelectKBest(mutual_info_classif, k=15)
sel_five_cols.fit(X_train, y_train)
c=X_train.columns[sel_five_cols.get_support()]
X_train3 = X_train[X_train.columns.intersection(c)]
X_test3=X_test[X_test.columns.intersection(c)]

In [10]:
sel_five_cols = SelectKBest(mutual_info_classif, k=15)
sel_five_cols.fit(X, y)
lb=X.columns[sel_five_cols.get_support()]
X1 = X[X.columns.intersection(lb)]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
rf4=ensemble.RandomForestClassifier(bootstrap=True, criterion='gini', max_depth=40,
                       min_samples_leaf=1, min_samples_split=2,
                       n_estimators=1577, max_features='sqrt',class_weight='balanced_subsample',random_state=0).fit(X1,y)
print(roc_auc_score(y, rf4.predict_proba(X1)[:, 1]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


1.0


In [12]:
prediction=rf4.predict(X1)
print(confusion_matrix(y,prediction))
print(accuracy_score(y,prediction))
print(classification_report(y,prediction))

[[  948     0]
 [    0 15435]]
1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       948
           1       1.00      1.00      1.00     15435

    accuracy                           1.00     16383
   macro avg       1.00      1.00      1.00     16383
weighted avg       1.00      1.00      1.00     16383



In [13]:
from xgboost import XGBClassifier

xgb_model1 = XGBClassifier(eta=0.15, colsample_bytree=0.725, eval_metric='auc', max_depth=20,gamma=0.05,min_child_weight=0.8,
              n_estimators=1000, scale_pos_weight=0.08, reg_lambda=2,reg_alpha=2.5,subsample=0.5,random_state=0).fit(X1, y)

# predict
xgb_y_predict = xgb_model1.predict(X1)
xgb_score = accuracy_score(xgb_y_predict, y)

print('Accuracy score is:', xgb_score)
print(roc_auc_score(y, xgb_model1.predict_proba(X1)[:, 1]))
#print(roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:, 1]))

prediction=xgb_model1.predict(X1)
print(confusion_matrix(y,prediction))
print(accuracy_score(y,prediction))
print(classification_report(y,prediction))

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy score is: 0.9736922419581273
0.9979975233010624
[[  946     2]
 [  429 15006]]
0.9736922419581273
              precision    recall  f1-score   support

           0       0.69      1.00      0.81       948
           1       1.00      0.97      0.99     15435

    accuracy                           0.97     16383
   macro avg       0.84      0.99      0.90     16383
weighted avg       0.98      0.97      0.98     16383



In [14]:
Xt=df_test.drop(['Id'],axis=1)
Xt=np.array(Xt)
Xt=pd.DataFrame(Xt)
Xt

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,37733.0,1.0,1.77,118603.0,1.0,0.0,118602.0,118097.0,1.0,0.0,...,13881.0,117941.0,117887.0,1.0,117885.0,1.0,1.0,1.0,1.0,1.0
1,312129.0,1.0,3.54,118052.0,1.0,0.0,117961.0,290919.0,1.0,4.0,...,14638.0,118992.0,290919.0,1.0,118321.0,1.0,1.0,1.0,7.0,1.0
2,24884.0,1.0,23.01,118300.0,1.0,0.0,117961.0,302830.0,1.0,0.0,...,770.0,119181.0,4673.0,1.0,128230.0,1.0,1.0,1.0,14.0,1.0
3,4674.0,1.0,1.77,119091.0,1.0,0.0,119062.0,118036.0,1.0,9.0,...,16752.0,143531.0,290919.0,1.0,117905.0,1.0,1.0,1.0,81.0,1.0
4,68725.0,1.0,3.54,118300.0,1.0,0.0,117961.0,171056.0,1.0,0.0,...,4945.0,118360.0,118638.0,1.0,118636.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16380,971.0,1.0,1.77,118300.0,1.0,0.0,117961.0,310997.0,1.0,0.0,...,4864.0,123055.0,118424.0,1.0,120344.0,1.0,1.0,1.0,44.0,1.0
16381,105802.0,1.0,1.77,118327.0,1.0,0.0,117961.0,117906.0,1.0,0.0,...,32457.0,118320.0,290919.0,1.0,118321.0,1.0,1.0,1.0,4.0,1.0
16382,43096.0,1.0,3.54,117962.0,1.0,0.0,117961.0,118368.0,1.0,0.0,...,2014.0,117904.0,117887.0,1.0,118368.0,1.0,1.0,1.0,1.0,1.0
16383,31220.0,1.0,1.77,118386.0,1.0,0.0,117961.0,117906.0,1.0,1.0,...,1249.0,118910.0,290919.0,1.0,118321.0,1.0,1.0,1.0,2.0,2.0


In [15]:
Xt1 = Xt[Xt.columns.intersection(lb)]
#Xt1=np.array(Xt1)
print(Xt1)

             0         3    4         6         7    8    10        12  \
0       37733.0  118603.0  1.0  118602.0  118097.0  1.0  1.0  117888.0   
1      312129.0  118052.0  1.0  117961.0  290919.0  1.0  1.0  118322.0   
2       24884.0  118300.0  1.0  117961.0  302830.0  1.0  1.0  128231.0   
3        4674.0  119091.0  1.0  119062.0  118036.0  1.0  1.0  117908.0   
4       68725.0  118300.0  1.0  117961.0  171056.0  1.0  1.0  118639.0   
...         ...       ...  ...       ...       ...  ...  ...       ...   
16380     971.0  118300.0  1.0  117961.0  310997.0  1.0  1.0  120346.0   
16381  105802.0  118327.0  1.0  117961.0  117906.0  1.0  1.0  118322.0   
16382   43096.0  117962.0  1.0  117961.0  118368.0  1.0  1.0  118486.0   
16383   31220.0  118386.0  1.0  117961.0  117906.0  1.0  1.0  118322.0   
16384   41642.0  118052.0  1.0  117961.0  172635.0  1.0  1.0  117908.0   

             13       14        15        16        18   19   23  
0      2.453740  13881.0  117941.0  117887.0

In [16]:
preds=0.85*rf4.predict_proba(Xt1)+0.15*xgb_model1.predict_proba(Xt1)
preds

array([[9.57807437e-02, 9.04219262e-01],
       [2.79224224e-01, 7.20775786e-01],
       [1.35623068e-03, 9.98643775e-01],
       ...,
       [6.16603345e-04, 9.99383403e-01],
       [4.57136639e-02, 9.54286342e-01],
       [1.23869689e-01, 8.76130319e-01]])

In [17]:
pr=0.85*rf4.predict(Xt1)+0.15*xgb_model1.predict(Xt1)
pr

array([1., 1., 1., ..., 1., 1., 1.])

In [18]:
yt=pd.DataFrame(pr,dtype=int)
yt

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version, passing float-dtype values and an integer dtype to DataFrame will retain floating dtype if they cannot be cast losslessly (matching Series behavior). To retain the old behavior, use DataFrame(data).astype(dtype)
  """Entry point for launching an IPython kernel.


,0
0,1
1,1
2,1
3,1
4,1
...,...
16380,1
16381,1
16382,1
16383,1


In [19]:
yt.value_counts()

1    15164
0     1221
dtype: int64

In [20]:
y

,0
0,1
1,1
2,1
3,1
4,1
...,...
16378,1
16379,1
16380,1
16381,1


In [21]:
y=pd.DataFrame(y)
y_final=pd.concat([y,yt])
y_final

,0
0,1
1,1
2,1
3,1
4,1
...,...
16380,1
16381,1
16382,1
16383,1


In [22]:
y_final.value_counts()

1    30599
0     2169
dtype: int64

In [23]:
X_f=pd.concat([X,Xt],ignore_index=True)
X_f

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,15006.0,1.0,8.85,118213.0,1.0,0.0,118212.0,117906.0,1.0,5.0,...,7399.0,123201.0,290919.0,1.0,117905.0,1.0,1.0,1.0,1.0,1.0
1,4674.0,1.0,1.77,118327.0,1.0,0.0,117961.0,117906.0,1.0,0.0,...,7644.0,120318.0,290919.0,1.0,117905.0,1.0,1.0,1.0,1.0,1.0
2,80176.0,1.0,1.77,118343.0,1.0,0.0,117961.0,117906.0,2.0,2.0,...,4004.0,118856.0,290919.0,1.0,117905.0,1.0,1.0,1.0,1.0,1.0
3,83045.0,1.0,1.77,117903.0,2.0,0.0,117902.0,117906.0,1.0,0.0,...,7393.0,120428.0,290919.0,1.0,117905.0,1.0,1.0,1.0,7.0,1.0
4,42679.0,1.0,1.77,117962.0,1.0,0.0,117961.0,130085.0,1.0,5.0,...,44022.0,122215.0,290919.0,1.0,120172.0,1.0,2.0,1.0,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32763,971.0,1.0,1.77,118300.0,1.0,0.0,117961.0,310997.0,1.0,0.0,...,4864.0,123055.0,118424.0,1.0,120344.0,1.0,1.0,1.0,44.0,1.0
32764,105802.0,1.0,1.77,118327.0,1.0,0.0,117961.0,117906.0,1.0,0.0,...,32457.0,118320.0,290919.0,1.0,118321.0,1.0,1.0,1.0,4.0,1.0
32765,43096.0,1.0,3.54,117962.0,1.0,0.0,117961.0,118368.0,1.0,0.0,...,2014.0,117904.0,117887.0,1.0,118368.0,1.0,1.0,1.0,1.0,1.0
32766,31220.0,1.0,1.77,118386.0,1.0,0.0,117961.0,117906.0,1.0,1.0,...,1249.0,118910.0,290919.0,1.0,118321.0,1.0,1.0,1.0,2.0,2.0


In [24]:
mutual_info1 = mutual_info_classif(X_f, y_final)
mutual_info1 = pd.Series(mutual_info)
mutual_info1.index = X_f.columns
mutual_info1.sort_values(ascending=False)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


13    0.068488
14    0.038228
15    0.018512
7     0.017058
0     0.015676
3     0.011485
12    0.011158
18    0.008649
16    0.007334
6     0.005202
19    0.004999
10    0.004114
20    0.003293
17    0.002473
4     0.002256
21    0.002127
8     0.001813
9     0.001527
23    0.000999
22    0.000412
5     0.000410
1     0.000000
11    0.000000
2     0.000000
dtype: float64

In [25]:
X_final=pd.concat([X1,Xt1],ignore_index=True)
X_final

,0,3,4,6,7,8,10,12,13,14,15,16,18,19,23
0,15006.0,118213.0,1.0,118212.0,117906.0,1.0,1.0,117908.0,-0.218055,7399.0,123201.0,290919.0,117905.0,1.0,1.0
1,4674.0,118327.0,1.0,117961.0,117906.0,1.0,1.0,117908.0,0.472692,7644.0,120318.0,290919.0,117905.0,1.0,1.0
2,80176.0,118343.0,1.0,117961.0,117906.0,2.0,2.0,117908.0,-0.261004,4004.0,118856.0,290919.0,117905.0,1.0,1.0
3,83045.0,117903.0,2.0,117902.0,117906.0,1.0,1.0,117908.0,-0.374357,7393.0,120428.0,290919.0,117905.0,1.0,1.0
4,42679.0,117962.0,1.0,117961.0,130085.0,1.0,1.0,120173.0,1.000000,44022.0,122215.0,290919.0,120172.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32763,971.0,118300.0,1.0,117961.0,310997.0,1.0,1.0,120346.0,1.000000,4864.0,123055.0,118424.0,120344.0,1.0,1.0
32764,105802.0,118327.0,1.0,117961.0,117906.0,1.0,1.0,118322.0,0.580688,32457.0,118320.0,290919.0,118321.0,1.0,1.0
32765,43096.0,117962.0,1.0,117961.0,118368.0,1.0,1.0,118486.0,1.000000,2014.0,117904.0,117887.0,118368.0,1.0,1.0
32766,31220.0,118386.0,1.0,117961.0,117906.0,1.0,1.0,118322.0,-2.374507,1249.0,118910.0,290919.0,118321.0,1.0,2.0


In [26]:
X_final.shape,y_final.shape

((32768, 15), (32768, 1))

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
random_state=check_random_state(0)
permutation=random_state.permutation(X.shape[0])
X_final=np.array(X_final)
y_final=np.array(y_final)
X_final = X_final[permutation]
y_final=y_final[permutation]
X_final = X_final.reshape((X_final.shape[0], -1))
X_final=pd.DataFrame(X_final)
y_final=pd.DataFrame(y_final)
Xf_train, Xf_test, yf_train, yf_test = train_test_split(X_final, y_final, test_size=0.30,stratify=y_final, random_state=42)

In [28]:
rf5=ensemble.RandomForestClassifier(bootstrap=True, criterion='gini', max_depth=None,
                       min_samples_leaf=1, min_samples_split=2,
                       n_estimators=1577, max_features='sqrt',class_weight='balanced',random_state=0).fit(Xf_train,yf_train)
print(roc_auc_score(yf_train, rf5.predict_proba(Xf_train)[:, 1]))
print(roc_auc_score(yf_test, rf5.predict_proba(Xf_test)[:, 1]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


1.0
0.8892989224485327


In [29]:
prediction=rf5.predict(Xf_test)
print(confusion_matrix(yf_test,prediction))
print(accuracy_score(yf_test,prediction))
print(classification_report(yf_test,prediction))

[[  65  219]
 [   5 4626]]
0.9544252288911496
              precision    recall  f1-score   support

           0       0.93      0.23      0.37       284
           1       0.95      1.00      0.98      4631

    accuracy                           0.95      4915
   macro avg       0.94      0.61      0.67      4915
weighted avg       0.95      0.95      0.94      4915



In [30]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
#criterion = ['gini','entropy']
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 60, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [0.0001, 0.001,0.01,2,3,4]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4]
# Method of selecting samples for training each tree
bootstrap = [True]
class_weight=['balanced','balanced_subsample']
# Create the random grid
random_grid = {'class_weight':class_weight,
               'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rfcv = ensemble.RandomForestClassifier(criterion='gini',random_state=0)
rf_random = RandomizedSearchCV(estimator = rfcv, param_distributions = random_grid, scoring='roc_auc', n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(Xf_train, yf_train)
print(rf_random.best_estimator_)
print(rf_random.best_params_)

In [31]:
rf6=ensemble.RandomForestClassifier(bootstrap=True, criterion='gini', max_depth=30,
                       min_samples_leaf=1, min_samples_split=4,
                       n_estimators=900, max_features='sqrt',class_weight='balanced_subsample',random_state=0).fit(Xf_train,yf_train)
print(roc_auc_score(yf_train, rf6.predict_proba(Xf_train)[:, 1]))
print(roc_auc_score(yf_test, rf6.predict_proba(Xf_test)[:, 1]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


1.0
0.8893487246085019


In [32]:
xgb_model2 = XGBClassifier(eta=0.05, colsample_bytree=0.7, eval_metric='auc', max_depth=60,
              n_estimators=1500, scale_pos_weight=0.038, reg_lambda=2,reg_alpha=2.5, subsample=0.5,random_state=0).fit(Xf_train, yf_train)

# predict
xgb_y_predict = xgb_model2.predict(Xf_test)
xgb_score = accuracy_score(xgb_y_predict, yf_test)

print('Accuracy score is:', xgb_score)
print(roc_auc_score(yf_train, xgb_model2.predict_proba(Xf_train)[:, 1]))
print(roc_auc_score(yf_test, xgb_model2.predict_proba(Xf_test)[:, 1]))

prediction=xgb_model2.predict(Xf_test)
print(confusion_matrix(yf_test,prediction))
print(accuracy_score(yf_test,prediction))
print(classification_report(yf_test,prediction))

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy score is: 0.8374364191251271
0.9864700378708466
0.8621863984598587
[[ 192   92]
 [ 707 3924]]
0.8374364191251271
              precision    recall  f1-score   support

           0       0.21      0.68      0.32       284
           1       0.98      0.85      0.91      4631

    accuracy                           0.84      4915
   macro avg       0.60      0.76      0.62      4915
weighted avg       0.93      0.84      0.87      4915



In [33]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]
max_depth = [int(x) for x in np.linspace(20, 60, num = 11)]
colsample_bytree=[0.65,0.7,0.75,0.8]
learning_rate=[0.05,0.15,0.1,0.2]
min_child_weight=[1,3,5]
subsample=[0.5,0.7,0.9]
scale_pos_weight=[0.05,0.08,0.1,0.7,0.9]
reg_lambda=[1.5,2,2.5]
reg_alpha=[2,2.5,3]

random_grid2 = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'colsample_bytree':colsample_bytree,
               'learning_rate':learning_rate,
               'min_child_weight': min_child_weight,
               'subsample': subsample,
               'reg_lambda':reg_lambda,
               'reg_alpha':reg_alpha,
               'scale_pos_weight':scale_pos_weight}
modrs = XGBClassifier(objective='binary:logistic',eval_metric='auc')
mod_random = RandomizedSearchCV(estimator = modrs, param_distributions = random_grid2,error_score='raise', n_iter =100,cv=3 ,scoring='roc_auc', verbose=2, random_state=42, n_jobs = -1)
mod_random.fit(Xf_train, yf_train)
print(mod_random.best_estimator_)
print(mod_random.best_params_)

In [34]:
xgb_model3 = XGBClassifier(eta=0.05, colsample_bytree=0.65, eval_metric='auc', max_depth=52,min_child_weight= 1,
              n_estimators=700, scale_pos_weight=0.8, reg_lambda=2,reg_alpha=2.5, subsample=0.5,random_state=0).fit(Xf_train, yf_train)

# predict
xgb_y_predict = xgb_model3.predict(Xf_test)
xgb_score = accuracy_score(xgb_y_predict, yf_test)

print('Accuracy score is:', xgb_score)
print(roc_auc_score(yf_train, xgb_model3.predict_proba(Xf_train)[:, 1]))
print(roc_auc_score(yf_test, xgb_model3.predict_proba(Xf_test)[:, 1]))

prediction=xgb_model3.predict(Xf_test)
print(confusion_matrix(yf_test,prediction))
print(accuracy_score(yf_test,prediction))
print(classification_report(yf_test,prediction))

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy score is: 0.9593082400813835
0.9991687873299937
0.8754379548724
[[  95  189]
 [  11 4620]]
0.9593082400813835
              precision    recall  f1-score   support

           0       0.90      0.33      0.49       284
           1       0.96      1.00      0.98      4631

    accuracy                           0.96      4915
   macro avg       0.93      0.67      0.73      4915
weighted avg       0.96      0.96      0.95      4915



In [35]:
xgb_modelf = XGBClassifier(eta=0.05, colsample_bytree=0.65, eval_metric='auc', max_depth=52,min_child_weight= 1,
              n_estimators=700, scale_pos_weight=0.8, reg_lambda=2,reg_alpha=2.5, subsample=0.5,random_state=0).fit(X_final, y_final)

# predict
xgb_y_predict = xgb_modelf.predict(X_final)
xgb_score = accuracy_score(xgb_y_predict, y_final)

print('Accuracy score is:', xgb_score)
print(roc_auc_score(y_final, xgb_modelf.predict_proba(X_final)[:, 1]))
#print(roc_auc_score(yf_test, xgb_modelf.predict_proba(Xf_test)[:, 1]))

prediction=xgb_modelf.predict(X_final)
print(confusion_matrix(y_final,prediction))
print(accuracy_score(y_final,prediction))
print(classification_report(y_final,prediction))

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy score is: 0.9848013184398462
0.9992658063828304
[[  702   246]
 [    3 15432]]
0.9848013184398462
              precision    recall  f1-score   support

           0       1.00      0.74      0.85       948
           1       0.98      1.00      0.99     15435

    accuracy                           0.98     16383
   macro avg       0.99      0.87      0.92     16383
weighted avg       0.98      0.98      0.98     16383



In [36]:
solution=pd.DataFrame({'Id':df_test.Id,'Y':preds[:,1]})
solution.head()

,Id,Y
0,16384,0.904219
1,16385,0.720776
2,16386,0.998644
3,16387,0.989765
4,16388,0.984964


In [37]:
solution.to_csv("submission.csv", index = False)